In [4]:
print("OK")

OK


In [5]:
%pwd

'd:\\Ashok\\AI\\2082_projects\\PRACTISES\\Medical_CHATBOT\\research'

In [6]:
import os
os.chdir(r"D:\Ashok\AI\2082_projects\PRACTISES\Medical_CHATBOT")
%pwd

'D:\\Ashok\\AI\\2082_projects\\PRACTISES\\Medical_CHATBOT'

In [7]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader #for loading pdf files
from langchain.text_splitter import RecursiveCharacterTextSplitter  #for chunking

In [8]:
# #load the pdf

# def load_pdf_file(data):
#     loader = DirectoryLoader(data, glob="**/*.pdf", loader_cls =PyPDFLoader)
    
#     documents = loader.load()
    
#     return documents


In [9]:
# extracted_data = load_pdf_file(data= 'Data/')

In [10]:
# extracted_data

## Perform CHUNKING

In [11]:
# # split the data into chunks
# def text_split(extracted_data):
#     text_splitter = RecursiveCharacterTextSplitter(
#         chunk_size=500,
#         chunk_overlap=20,
#     )
#     text_chunks = text_splitter.split_documents(extracted_data)
#     return text_chunks

In [12]:
# text_chunks = text_split(extracted_data)
# print("length of text chunks: ", len(text_chunks))

## Embedding to vector DB with pinecone

In [23]:
from langchain.embeddings import HuggingFaceEmbeddings

In [24]:
#Download the embedding model from hugging face

def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [25]:
embeddings = download_hugging_face_embeddings()

C:\Users\ashok\AppData\Local\Temp\ipykernel_18044\2396306128.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')


In [16]:
# query_result = embeddings.embed_query("Hello World")
# print("Length of query result: ", len(query_result))

In [17]:
# query_result

In [18]:
from dotenv import load_dotenv

load_dotenv()
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
GEMINI_API_KEY = os.environ.get('GEMINI_API_KEY')

In [20]:
# from pinecone.grpc import PineconeGRPC as Pinecone
# from pinecone import ServerlessSpec
# import os

# pc = Pinecone(api_key=PINECONE_API_KEY)

# index_name = "medical-chatbot"

# pc.create_index(
#     name=index_name,
#     dimension=384,  # 384 is the dimension of the all-MiniLM-L6-v2 model
#     metric="cosine",
#     spec = ServerlessSpec(
#         cloud="aws",
#         region="us-east-1"
#     )
# )     

In [63]:
# #Embed each chunk and upsert the embedding into oyur PInecone index
# from langchain_pinecone import PineconeVectorStore

# docsearch = PineconeVectorStore.from_documents(
#     documents = text_chunks, # list of documents to be embedded
#     index_name = index_name, # name of the index to upsert the embeddings into
#     embedding = embeddings, # embedding model to use for embedding the documents
# )

In [26]:
#Load existing index
from langchain_pinecone import PineconeVectorStore
#Embed each chunk and embeddings into your Pinecone index
docsearch = PineconeVectorStore.from_existing_index(
    index_name = index_name, # name of the index to upsert the embeddings into
    embedding = embeddings, # embedding model to use for embedding the documents
)


In [27]:
docsearch

In [28]:
retriever = docsearch.as_retriever(search_type='similarity', search_kwargs={"k": 3}) # k is the number of documents to retrieve

In [29]:
retrieved_docs = retriever.invoke("What is Acne?")

In [30]:
retrieved_docs

[Document(id='fbf89ce8-61a6-4719-9ad9-939f3b1dac0a', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='1d3a4765-3256-4256-b75a-10fa4f7cbd05', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 'page_label': '39', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed.(Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM - 0001 to 0432 - 

## Connecting LLM with the Knowledge base

In [31]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=1000,
    timeout=None,
    max_retries=2,
    google_api_key=GEMINI_API_KEY
)

In [32]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use only the retrieved context to answer the question. "
    "If the context does not contain the answer, say 'I don't know based on the provided context.' "
    "Do not attempt to answer from your own knowledge. "
    "Use three sentences maximum and keep the answer concise."
    "\n\n"
    "Context:\n{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{context}"),   
    ]
)

In [33]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [34]:
response = rag_chain.invoke({"input": "What is the treatmenet for Acne?"})
print(response['answer'])

Treatment for mild noninflammatory acne consists of reducing the formation of new comedones with topical tretinoin, benzoyl peroxide, adapalene, or salicylic acid. Tretinoin is especially effective because it increases turnover of skin cells. When complicated by inflammation, topical antibiotics may be added to the treatment.


In [112]:
query = "What is CPU"
docs = retriever.get_relevant_documents(query)

if not docs:
    print("No relevant medical documents found.")
else:
    response = rag_chain.invoke({"input": query})
    print(response['answer'])



I'm sorry, but the question you're asking is missing. Please provide the question so I can answer it using the provided context.
